<a href="https://colab.research.google.com/github/jarora-dev/DL-Image-Caption-Generator/blob/main/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Caption Generator using CNN and LSTM

In [2]:
# Import Libraries
import pandas as pd
import numpy as np